In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json 
import os

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import Bounds
from scipy.optimize import minimize

In [3]:
import seaborn as sns
sns.set(
    style='whitegrid', # стиль figure
    font_scale=2, # размер шрифта
    rc={'lines.linewidth': 3, # ширина линий
        'text.usetex' : True} # использовать tex
)

In [4]:
from data_utils import read_data, normalization, plot_results
from opt_utils import func, fit_curve 
from opt_utils import f_standard, get_x0_standard, unpack_standard
from opt_utils import f_exp, get_x0_exp, unpack_exp

In [5]:
bounds_standard = Bounds(
    np.array([0, 0, 0, 0, 0, 0], dtype=np.float64), 
    np.array([np.inf, np.inf, np.inf, np.inf, np.inf, np.inf], dtype=np.float64)
#     [0, 0, 0, 0, 0, 0],
#     [None, None, None, None, None, None]
)

# from scipy.optimize import LinearConstraint
# linear_constraint = LinearConstraint([[1, 1, 1, 0, 0, 0], 
#                                       [0, 0, 0, 1, -1, 0]], 
#                                      [1, 1], 
#                                      [0, np.inf])

cons_standard = [
    {
        'type': 'eq',
        'fun': lambda x: np.array([x[0] + x[1] + x[2] - 1]),
        'jac': lambda x: np.array([1.0, 1.0, 1.0, 0, 0, 0])
    },
    {
        'type': 'ineq',
        'fun': lambda x: np.array([x[3] - x[4]]),
        'jac': lambda x: np.array([0, 0, 0, 1.0, -1.0, 0])
    },
]

bounds_exp = Bounds(
    np.array([0, 0], dtype=np.float64), 
    np.array([np.inf, np.inf], dtype=np.float64)
)

cons_exp = []

## Собираем все дериктории стеков

In [7]:
source_path = 'soil_CFs'

In [11]:
dirs = [d for d in os.listdir(source_path) if not d.endswith('.png')]
results = []
# for cdir in ['5', '13']: 
#     dirs.remove(cdir)
#     dirs.extend([os.path.join(cdir, d) for d in os.listdir(os.path.join(source_path, cdir))])
# dirs.remove('.ipynb_checkpoints')

In [12]:
dirs

['CF11_NN',
 'CF11',
 'CF9_NN',
 'CF12_NN',
 'REV1_1_NN',
 'CF3',
 'REV1_1',
 'CF12',
 'CF14_NN',
 'CF9',
 'REV2_1',
 'REV2_1_NN',
 'CF3_NN',
 'CF14']

In [ ]:
for dir_name in dirs:
    data_path = os.path.join(source_path, dir_name) 
    data = read_data(data_path)
    
    dir_path = os.path.join(data_path, 'results')
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    print('Current directory: {}'.format(dir_path))
    
    # Добавим в данные корр функции, усредненные по всем направлениям
    mean_data = dict()
    for tag, v in data.items():
        tag_edited = tag[:-1]
        if tag_edited not in mean_data:
            mean_data[tag_edited] = np.array(v)
        else:
            mean_data[tag_edited] += np.array(v)
    for tag, v in mean_data.items():
        mean_data[tag] = list(v / 3)
    data.update(mean_data)
    
    # Для каждого тега сделаем фит
    for tag in tqdm(data):
        phi = data[tag][0] 
        exp_data = normalization(data, tag=tag)
        n = exp_data.shape[0]

        scores = dict()
        # Семплируем много стартовых точек, чтобы получить наилучшее решение
        for i in range(50):
            # Два способа зафитить кривую - чисто экспоненциальный или две экспоненты + квадратичное слагаемое
#             if tag.startswith('L2'):
#                 result = fit_curve(exp_data, f_exp, get_x0_exp, 
#                                    unpack_exp, 
#                                    bounds_exp, 
#                                    cons_exp, 
#                                    phi=phi
#                                   )
#             else:
            result = fit_curve(exp_data, f_standard, get_x0_standard,
                               unpack_standard, 
                               bounds_standard, 
                               cons_standard
                              )
            if result['success']:
                scores[result['SSE']] = result

        best = min(scores) 
        result = scores[best]
        result['tag'] = tag
        result['dir_name'] = dir_name
        labels_mapping = {
            'fitted': 'Fitted',
            'exp_data': 'Data'
        }
        plot_results(
            result, 
            labels_mapping=labels_mapping,
            tag=tag, 
            dir_path=dir_path
        )
        results.append(result)

In [18]:
def plot_results(
        result, 
        ax, 
        labels_mapping=None, 
        labels=None, 
        tag=None, 
        x_up=1, 
        x_down=-0.1, 
        dpi=400
):
    if labels is None:
        labels = ['fitted', 'exp_data']
    
    xlim = 0
    for label in labels:
        y = result[label]
        r = np.arange(0, len(y))
        xlim = max(xlim, len(y))
        label = label if labels_mapping is None else labels_mapping[label]
        ax.plot(r, y, label=label)
    # plt.plot(r, z, label='Data S2')
#     plt.plot(r, y, label='Fitted')
    ax.legend(loc='best')
    if tag is not None:
        ax.set_title(r'${corr_func}_{{ {direction} }}$'.format(corr_func=tag[0], direction=tag[1:]))
    ax.set_ylim([x_down, x_up])
    ax.set_xlim([0, xlim])    

In [19]:
def print_params(ax, result):
    ys = [0.05, 0.23, 0.41, 0.59, 0.77, 0.95]
    seq = ['a', 'b', 'c', 'a1', 'a2', 'a3'][::-1]
    result = unpack_standard(result['x'])
    for y, k in zip(ys, seq):
        v = result[k]
        if (len(k) > 1):
            k = k[0] + '_' + k[1:]
        s = r'${} = {:.4f}$'.format(k, v)
        ax.text(0.25, y, s, fontsize=50)

In [20]:
stacks_mapping = {
    '1' : '1', 
    '10' : '2', 
    '11' : '3', 
    '12' : '4', 
    '13/down' : '5', 
    '13/upper' : '6',
    '14' : '7', 
    '2' : '8', 
    '3' : '9', 
    '4' : '10', 
    '5/down_part900' : '11',
    '5/upper_part900' : '12', 
    '6' : '13', 
    '7' : '14', 
    '8' : '15', 
    '9'  : '16'
}

In [76]:
tags = ['C2', 'L2', 'S2', 'SS2']

In [22]:
stacks = list(stacks_mapping.keys())

In [39]:
import matplotlib.image as mpimg

In [63]:
filtered_results = {r['tag']: r 
                    for r in results 
                    if (r['dir_name'] == stack)} # and (r['tag'][-1] in ['X', 'Y', 'Z'])}

In [78]:
keys = sorted(filtered_results.keys())
keys

['C2',
 'C2X',
 'C2Y',
 'C2Z',
 'L2',
 'L2X',
 'L2Y',
 'L2Z',
 'S2',
 'S2X',
 'S2Y',
 'S2Z',
 'SS2',
 'SS2X',
 'SS2Y',
 'SS2Z']

In [86]:
for stack in stacks:
    fig = plt.figure(constrained_layout=True, figsize=(40, 100))

    gs = fig.add_gridspec(2 + 2 * len(tags), 4)

    ax = fig.add_subplot(gs[0:2, 0:2])
    img=mpimg.imread('CFs/sample{}.png'.format(stacks_mapping[stack]))
    ax.axis('off')
    ax.imshow(img)

    ax = fig.add_subplot(gs[0:2, 2:4])
    for k in keys:
        x = filtered_results[k]['exp_data']
        ax.plot(np.arange(len(x)), x, '--', label=k)        
    ax.legend(loc='best')

    shift = 2

    for i, tag in enumerate(tags):
        for j, suff in enumerate(['', 'X', 'Y', 'Z']):
            ax = fig.add_subplot(gs[shift+0+2*i:shift+1+2*i, 0+j:1+j])
            plot_results(filtered_results[tag + suff], ax, tag=tag + suff, labels_mapping=labels_mapping)

            ax = fig.add_subplot(gs[shift+1+2*i:shift+2+2*i, 0+j:1+j])
            print_params(ax, filtered_results[tag + suff])
            ax.axis('off')

#     plt.show()
    plt.savefig('compiled/{}.png'.format(stacks_mapping[stack]))
    plt.close('all')

In [93]:
SSEs = {(stacks_mapping[r['dir_name']], r['tag']): r['SSE'] for r in results if r['tag'][-1] not in ['X', 'Y', 'Z']}

In [95]:
sorted(SSEs.items(), key=lambda item: item[1], reverse=True)

[(('13', 'C2'), 0.02299206915591812),
 (('10', 'C2'), 0.022144128460825606),
 (('4', 'C2'), 0.021127634252452183),
 (('15', 'C2'), 0.021122430317305107),
 (('10', 'S2'), 0.018049298170469973),
 (('1', 'SS2'), 0.01657650159821126),
 (('8', 'SS2'), 0.016317642613046062),
 (('11', 'C2'), 0.015982738666142415),
 (('8', 'C2'), 0.015764876337991177),
 (('15', 'SS2'), 0.014660279943899516),
 (('2', 'SS2'), 0.013949518755376733),
 (('10', 'SS2'), 0.013937972029981078),
 (('9', 'SS2'), 0.013861118602478366),
 (('9', 'C2'), 0.013825565154590245),
 (('14', 'C2'), 0.012986915230810811),
 (('8', 'S2'), 0.012665226676725927),
 (('13', 'SS2'), 0.012196824622479011),
 (('14', 'SS2'), 0.011377122465037802),
 (('4', 'SS2'), 0.011283205504484022),
 (('11', 'SS2'), 0.010829711680256638),
 (('3', 'SS2'), 0.010810325888442643),
 (('12', 'S2'), 0.009420375605373938),
 (('1', 'C2'), 0.008328899161403287),
 (('3', 'C2'), 0.0077380356212790585),
 (('7', 'SS2'), 0.00739048645759463),
 (('16', 'SS2'), 0.007156611

In [20]:
X = []
Y = []
for r in results:
    X.append(np.array(r['x']).reshape(1, -1))
    Y.append('{}_{}'.format(r['dir_name'], r['tag']))
X = np.concatenate(X, axis=0)

In [21]:
import pickle
with open('viz_data_new.pkl', 'wb') as f:
    pickle.dump((X, Y), f)

In [22]:
with open('data_new.pkl', 'wb') as f:
    pickle.dump(results, f)